In [ ]:
!git clone -b video https://github.com/jctestud/pix2pixHD.git

In [2]:
!pip install netCDF4
!pip install basemap

     |████████████████████████████████| 4.7 MB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 238 kB 71.6 MB/s eta 0:00:01
     |████████████████████████████████| 862 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 30.5 MB 57.6 MB/s eta 0:00:01
     |████████████████████████████████| 16.8 MB 63.9 MB/s eta 0:00:01
     |████████████████████████████████| 7.7 MB 13.2 MB/s eta 0:00:01
     |████████████████████████████████| 219 kB 31.0 MB/s eta 0:00:01
  Created wheel for pyshp: filename=pyshp-2.1.3-py3-none-any.whl size=37263 sha256=ab89686e544b2a15abb24b1afd0b1cc2485f3aa1c57ff5ecc06c2f2b1c94eb89
  Stored in directory: /home/jovyan/.cache/pip/wheels/1f/1b/b5/54affbefc8a7e2bdf1da000fc576b8a1c91338f1f327a04f4c
Successfully built pyshp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the p

In [ ]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import glob
from datetime import datetime, date, timedelta 
from pathlib import Path
from mpl_toolkits.basemap import Basemap

In [ ]:
files = glob.glob(r'data/2021/*/*.nc')

for f in files:
    nc = Dataset(f)
    plt.close()
    lon = np.transpose(nc.variables['REFERENCE_POINT_LON'][:])
    lat = np.transpose(nc.variables['REFERENCE_POINT_LAT'][:])
    mean = lat.data;
    mean = mean[~np.isnan(mean)];
    mean = mean.mean();
    radiance = nc.variables['RADIANCE'][:]
    wavelength = nc.variables['WAVELENGTH'][:]
    wave_len = len(wavelength) / 2
    wave_len = int(wave_len)
    wave_len2 = len(wavelength[0])/ 2
    wave_len2 = int(wave_len2)
    sza = np.transpose(nc.variables['SOLAR_ZENITH_ANGLE'][:])
    O5s_ids = np.argwhere((135.0 <= wavelength[wave_len,wave_len2,:]) & (wavelength[wave_len,wave_len2,:] <= 137.0))
    O5s = np.nansum(radiance[:,:,O5s_ids],axis = 2)*.04
    O5s = np.transpose(O5s[:,:,0])
        
    LBH_ids = np.argwhere((141.0 < wavelength[wave_len,wave_len2,:]) & (wavelength[wave_len,wave_len2,:] < 153.0))
    LBHs = np.nansum(radiance[:,:,LBH_ids],axis = 2)*.04
    LBHs = np.transpose(LBHs[:,:,0])

    mp = Basemap(projection='geos',
            rsphere=(6378137.00,6356752.3142),
            resolution='c',
            lon_0=-45)

    x,y = mp(lon,lat)
    hour = f[40:42]
    minutes = f[43:45]
    day_num = f[36:39]
    if hour != "00":
        day_num.rjust(13 + len(day_num), '0') 
        year = 2020
        start_date = date(int(year), 1, 1)
        result_date = start_date + timedelta(days=int(day_num) - 1) 
        result = result_date.strftime("%Y-%m-%d")
        if 'CHA' in f:
            channel = "CHA"
    
        if 'CHB' in f:
            channel = "CHB"
    
        c_scheme = mp.pcolor(x,y,O5s,cmap = 'jet',vmin=0,vmax=40)
        mp.drawcoastlines()
   
        if mean > 0:
            plt.savefig("pix2pixHD/datasets/north_year/test_frames/" + result + "_" + hour + "_" + minutes + "_" + channel +"_1356.png")
        if mean < 0:
            plt.savefig("pix2pixHD/datasets/south_year/test_frames/" + result + "_" + hour + "_" + minutes + "_" + channel +"_1356.png")

        
    
    plt.close()